In [ ]:
# Installing necessary libraries needed to do the training (uncomment when needed)
"""
!pip install datasets
!pip install joblib
!pip install ray[tune]
"""

In [1]:
import re
import torch
import joblib

import pandas as pd

from collections import Counter
from datasets import Dataset
from preprocess import TextCleaner
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding

In [2]:
# loading the dataset
df = pd.read_csv('trials.csv')

In [3]:
# Removing descriptions that are shorter than 10 words as they are not relevant/carry enought info(see data_analysis_trials.ipynb)
df = df[df['description'].apply(lambda x: len(str(x).split()) >= 10)].reset_index(drop=True)

In [4]:
# Cleaning text using the clean_text function in preprocess.py
cleaner = TextCleaner()
df['cleaned_text'] = df['description'].apply(cleaner.clean_text)

In [5]:
# Apply label encoding to the labels available.
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'].tolist())

In [6]:
# Confirm the label encoding and find which label is assigned which integer
classes = [str(item) for item in label_encoder.classes_]
print(dict(zip(range(len(label_encoder.classes_)), classes)))

{0: 'ALS', 1: 'Dementia', 2: 'Obsessive Compulsive Disorder', 3: 'Parkinson’s Disease', 4: 'Scoliosis'}


In [7]:
# Split the dataset to train/test using 80% train and 20% test
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'])

In [8]:
# Confirm the distribution of labels in the dataset
labels_counter = Counter(test_df['label'])
print("labels counter: ", labels_counter)

labels counter:  Counter({1: 74, 0: 73, 2: 71, 4: 67, 3: 65})


In [9]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Convert datasets to tokenized format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

def tokenize_data(examples):
    return tokenizer(examples["cleaned_text"], truncation=True)

tokenized_train = train_dataset.map(tokenize_data, batched=True)
tokenized_test = test_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/1399 [00:00<?, ? examples/s]

Map:   0%|          | 0/350 [00:00<?, ? examples/s]

In [10]:
# Use cuda if available, otherwise; cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
# Load pre-trained DistilBERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)
model.to(device)

# Prepare data collator for padding sequences
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results_5epochs_v10",
    learning_rate=2e-05,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.05,
    report_to=None,
    eval_strategy='epoch',
    logging_strategy="epoch"
)

# Define Trainer object for training the model
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save the trained model
trainer.save_model('model')


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_27843/2463515744.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.635800,0.311765,0.911429,0.910737
2,0.196400,0.273623,0.917143,0.917029
3,0.098000,0.230500,0.940000,0.940098


In [15]:
# evaluate the model on the test dataset
trainer.evaluate()

{'eval_loss': 1.613584041595459,
 'eval_model_preparation_time': 0.0018,
 'eval_accuracy': 0.2057142857142857,
 'eval_f1': 0.09408575763240226,
 'eval_runtime': 1.6689,
 'eval_samples_per_second': 209.719,
 'eval_steps_per_second': 52.729}

In [ ]:
# Dump the trained model and tokenizer as joblib objects
joblib.dump(model, 'DistilBERT_model.joblib')
joblib.dump(tokenizer,'DistilBERT_tokenizer.joblib')

In [ ]:
from huggingface_hub import HfApi
import os

api = HfApi(token=os.getenv("HF_TOKEN"))
api.upload_folder(
    folder_path="model",
    repo_id="hghomeshi/DistilBERT_text_classification_trials",
    repo_type="model",
)